# RUN

In [2]:
import os
import subprocess
import pickle
from tqdm import tqdm

from Bio.PDB.PDBParser import PDBParser
import numpy as np
import pandas as pd
import os


In [3]:

### Define LigPlot+ environment here
components_cif = '/Users/kyoheikoyama/workspace/LigPlus/lib/params/components.cif' # Location of components.cif
ligplot_plus = '/Users/kyoheikoyama/workspace/LigPlus/lib/exe_mac/' # Location of your LigPlus executable folder
pdbdir = './pdb/'
path_DICT_PDBID_2_CDRS = "/Users/kyoheikoyama/workspace/tcrpred/analysis/analysis1_alldata/../DICT_PDB_Result/DICT_PDBID_2_CDRS.pickle"
path_DICT_PDBID_2_CHAINNAMES = "/Users/kyoheikoyama/workspace/tcrpred/analysis/analysis1_alldata/../DICT_PDB_Result/DICT_PDBID_2_CHAINNAMES.pickle"

LIGPLOT_ROOT = "/Users/kyoheikoyama/workspace/ligplottmp"


# Check

In [5]:
# PDBID = pid = '1ao7'
# PDBID = pid = '4ftv' # '1ao7'
PDBID = pid = '4GRM' # '1ao7'

# def get_structure_from_id(ent):
#     structure = parser.get_structure("PHA-L", "1FAT.pdb")
    
from Bio.PDB import *
import warnings
warnings.filterwarnings('ignore')

# print(PDBID)

def download_pdb(pdbid, pdir='./pdb/'):
    from Bio.PDB import PDBList
    pdbl = PDBList()
    pdbl.retrieve_pdb_file(pdbid, pdir=pdir, file_format='pdb')
    
def get_structure(pdbid, pdir='./pdb/'):
    pdbid = pdbid.upper()
    parser = PDBParser()
    structure = parser.get_structure(pdbid, f"./pdb/pdb{PDBID.lower()}.ent")
    return structure

def run_hbplus(filename):
    comlist = ['{}hbadd'.format(ligplot_plus), 
               filename, 
               components_cif, 
               '-wkdir', 
               f'{LIGPLOT_ROOT}/hbadd_hbplus_result/']
    print(comlist)
    subprocess.check_call(comlist, shell = False)
    file_prefix = filename.split('/')[-1].replace('.ent', '')

    if len(file_prefix)!=7:
        print(file_prefix, filename)
        assert False

    subprocess.check_call(['{}hbplus'.format(ligplot_plus), '-L', '-h', '2.90', '-d', '3.90', '-N', filename, '-wkdir', f'{LIGPLOT_ROOT}/hbadd_hbplus_result/'], shell = False)
    subprocess.check_call(['{}hbplus'.format(ligplot_plus), '-L', '-h', '2.70', '-d', '3.35', filename, '-wkdir', f'{LIGPLOT_ROOT}/hbadd_hbplus_result/'], shell = False)



In [4]:
download_pdb(pid)

Structure exists: './pdb/pdb4ftv.ent' 


In [6]:
run_hbplus(f'./pdb/pdb{pid}.ent')

['/Users/kyoheikoyama/workspace/LigPlus/lib/exe_mac/hbadd', './pdb/pdb4ftv.ent', '/Users/kyoheikoyama/workspace/LigPlus/lib/params/components.cif', '-wkdir', '/Users/kyoheikoyama/workspace/ligplottmp/hbadd_hbplus_result/']
  Opening PDB file [./pdb/pdb4ftv.ent] ...

    Searching for HET groups in PDB file ...

    Processing CONECT records ...

      Number of HET residues read in        =      10
      Number of atom records stored         =      60
      Number of CONECT details stored       =     100

  Opening HET Group Dictionary ...

    Scanning HET Group Dictionary ...
      Found match for: GOL x 10

      Number of dictionary entries read in  =       1
    Residue GOL. Atoms:      6 in PDB entry,      6 in dictionary   
                 Bonds:      5 in PDB entry,      5 in dictionary   
                    Number of equiv_atoms created = 18
                    Number of equiv_bonds created = 34
          Have a perfect match with the dictionary for GOL ...

  Opening HBPLUS

In [12]:
!ls /Users/kyoheikoyama/workspace/ligplottmp/hbadd_hbplus_result/ | grep {pid}

pdb4ftv.hhb
pdb4ftv.nnb


# Result

In [10]:
!cat {LIGPLOT_ROOT}/hbadd_hbplus_result/pdb1ao7.hhb | grep "1AO7D"

1AO7D1AO7/D0001-LYS N   A1AO7/A0058-GLU OE2  3.09MS  -1 8.00135.0 2.29136.8 148.5    69
1AO7D1AO7/D0068-LYS NZ  A1AO7/A0166-GLU OE1  2.92SS  -1 9.27171.2 1.91132.5 131.0   214
1AO7D1AO7/D0052-ASN ND2 A1AO7/A0166-GLU OE2  3.12SS  -110.68107.3 2.68133.6 135.5   215
1AO7D1AO7/D0030-GLN NE2 C1AO7/C0002-LEU O    3.13SM  -110.25157.0 2.19159.0 163.3   399
1AO7D1AO7/D0100-SER N   C1AO7/C0004-GLY O    2.89MM  -1 5.92137.9 2.07159.6 163.2   400
1AO7D1AO7/D0100-SER OG  C1AO7/C0004-GLY O    3.10SM  -1 5.92165.1 2.12135.1 138.9   401
1AO7D1AO7/D0031-SER OG  C1AO7/C0005-TYR OH   2.54SS  -110.25138.4 1.71124.3 116.5   403
1AO7D1AO7/D0002-GLU N   D1AO7/D0025-SER OG   2.89MS  23 5.57128.9 2.16133.1 143.1   409
1AO7D1AO7/D0025-SER N   D1AO7/D0002-GLU O    3.04MM  23 5.57130.8 2.29158.3 165.9   410
1AO7D1AO7/D0003-VAL N   D1AO7/D0002-GLU OE1  3.31MS   1 3.74139.6 2.49109.0 100.4   411
1AO7D1AO7/D0004-GLU N   D1AO7/D0023-THR O    2.72MM  19 5.10156.5 1.77152.0 159.5   412
1AO7D1AO7/D0023-THR N   D1AO7/D0

In [15]:
!cat {LIGPLOT_ROOT}/hbadd_hbplus_result/pdb{pid}.hhb | grep {pid.upper()}

4FTVA4FTV/A0003-HIS N   A4FTV/A0103-VAL O    3.21MM 100 5.39147.9 2.31129.8 139.4     1
4FTVA4FTV/A0103-VAL N   A4FTV/A0003-HIS O    2.93MM 100 5.39162.6 1.96143.7 149.3     2
4FTVA4FTV/A0003-HIS ND1 A4FTV/A0029-ASP OD1  2.67SS  26 5.92169.5 1.68138.9 138.5     3
4FTVA4FTV/A0004-SER N   A4FTV/A0029-ASP OD2  2.60MS  25 5.10145.6 1.72157.4 158.1     4
4FTVA4FTV/A0029-ASP N   A4FTV/A0004-SER O    2.90MM  25 5.10160.8 1.94137.2 143.7     5
4FTVA4FTV/A0004-SER OG  A4FTV/A0029-ASP OD2  3.32SS  25 5.10172.8 2.32134.6 136.6     6
4FTVA4FTV/A0005-MET N   A4FTV/A0101-CYS O    3.12MM  96 5.57152.8 2.20151.4 159.5     7
4FTVA4FTV/A0101-CYS N   A4FTV/A0005-MET O    3.14MM  96 5.57147.7 2.25140.2 147.9     8
4FTVA4FTV/A0006-ARG N   A4FTV/A0027-TYR O    3.08MM  21 5.20141.4 2.23125.7 137.4     9
4FTVA4FTV/A0027-TYR N   A4FTV/A0006-ARG O    2.76MM  21 5.20172.5 1.77168.9 171.4    10
4FTVA4FTV/A0006-ARG NE  A4FTV/A0113-TYR OH   2.60SS 107 9.11129.2 1.85101.4 118.7    11
4FTVA4FTV/A0007-TYR N   A4FTV/A0

In [25]:
LIGPLOT_ROOT

'/Users/kyoheikoyama/workspace/ligplottmp'

In [29]:
pdbid = pid

In [43]:
pd.set_option('display.max_columns', 500)


In [40]:
import sys
sys.path.append('./../streamlit/')
from streamlit_utils import get_df, cleanup, HHB_COLUMNS, read_hhb_file, read_hhb_text, process_hhb

root = '/Users/kyoheikoyama/workspace/ligplottmp/hbadd_hbplus_result/'
df = pd.read_table(f'{root}pdb{pdbid.lower()}.hhb', header=None)


In [46]:
def get_donoar_acceptor(pdbid):
    df = read_hhb_file(pdbid)
    data_donors_n_acceptors = [read_hhb_text(row.item()) for i, row in df.iterrows()]
    data_donors_n_acceptors = [a for a in data_donors_n_acceptors if a is not None]
    df_donor_acceptor = pd.DataFrame(data_donors_n_acceptors)
    df_donor_acceptor = process_hhb(df_donor_acceptor)
    return df_donor_acceptor

In [77]:
CDRS = [f'D00{ii}' for ii in range(90,103)]
CDRS += [f'E00{ii}' for ii in range(92,103)]

In [93]:
pdbid = '1ao7'
# pdbid = '4ftv'


# Others

In [97]:
pdbid = '1G6R'
download_pdb(pdbid)
run_hbplus(f'./pdb/pdb{pdbid}.ent')

Structure exists: './pdb/pdb1g6r.ent' 
['/Users/kyoheikoyama/workspace/LigPlus/lib/exe_mac/hbadd', './pdb/pdb1G6R.ent', '/Users/kyoheikoyama/workspace/LigPlus/lib/params/components.cif', '-wkdir', '/Users/kyoheikoyama/workspace/ligplottmp/hbadd_hbplus_result/']
  Opening PDB file [./pdb/pdb1G6R.ent] ...

    Searching for HET groups in PDB file ...

    Processing CONECT records ...

      Number of HET residues read in        =       0
      Number of atom records stored         =       0
      Number of CONECT details stored       =      36

  Opening HET Group Dictionary ...

    Scanning HET Group Dictionary ...

      Number of dictionary entries read in  =       0
  Opening output bonds file, /Users/kyoheikoyama/workspace/ligplottmp/hbadd_hbplus_result/hbadd.bonds ...
HBPLUS Hydrogen Bond Calculator v 3.2            Apr 25 20:47:12 JST 2023
(c) I McDonald, D Naylor, D Jones and J Thornton 1993 All Rights Reserved.

Configured for 100000 atoms and 24000 residues.

Criteria

Neighb

In [100]:
pdbid = '4L3E'
download_pdb(pdbid)
run_hbplus(f'./pdb/pdb{pdbid}.ent')
pdbid = '3QDG'
download_pdb(pdbid)
run_hbplus(f'./pdb/pdb{pdbid}.ent')

['/Users/kyoheikoyama/workspace/LigPlus/lib/exe_mac/hbadd', './pdb/pdb4L3E.ent', '/Users/kyoheikoyama/workspace/LigPlus/lib/params/components.cif', '-wkdir', '/Users/kyoheikoyama/workspace/ligplottmp/hbadd_hbplus_result/']
  Opening PDB file [./pdb/pdb4L3E.ent] ...

    Searching for HET groups in PDB file ...

    Processing CONECT records ...

      Number of HET residues read in        =       0
      Number of atom records stored         =       0
      Number of CONECT details stored       =      28

  Opening HET Group Dictionary ...

    Scanning HET Group Dictionary ...

      Number of dictionary entries read in  =       0
  Opening output bonds file, /Users/kyoheikoyama/workspace/ligplottmp/hbadd_hbplus_result/hbadd.bonds ...
HBPLUS Hydrogen Bond Calculator v 3.2            Apr 26 04:19:11 JST 2023
(c) I McDonald, D Naylor, D Jones and J Thornton 1993 All Rights Reserved.

Configured for 100000 atoms and 24000 residues.

Criteria

Neighbour interactions, max atom-atom separat

In [7]:
pdbid = PDBID = pid = '4GRM' # '1ao7'

download_pdb(pdbid)
run_hbplus(f'./pdb/pdb{pdbid}.ent')


['/Users/kyoheikoyama/workspace/LigPlus/lib/exe_mac/hbadd', './pdb/pdb4GRM.ent', '/Users/kyoheikoyama/workspace/LigPlus/lib/params/components.cif', '-wkdir', '/Users/kyoheikoyama/workspace/ligplottmp/hbadd_hbplus_result/']
  Opening PDB file [./pdb/pdb4GRM.ent] ...

    Searching for HET groups in PDB file ...

    Processing CONECT records ...

      Number of HET residues read in        =       0
      Number of atom records stored         =       0
      Number of CONECT details stored       =      40

  Opening HET Group Dictionary ...

    Scanning HET Group Dictionary ...

      Number of dictionary entries read in  =       0
  Opening output bonds file, /Users/kyoheikoyama/workspace/ligplottmp/hbadd_hbplus_result/hbadd.bonds ...
HBPLUS Hydrogen Bond Calculator v 3.2            Apr 26 16:28:02 JST 2023
(c) I McDonald, D Naylor, D Jones and J Thornton 1993 All Rights Reserved.

Configured for 100000 atoms and 24000 residues.

Criteria

Neighbour interactions, max atom-atom separat